# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
<div class="span5 alert alert-info">
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</div>

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.api as sms

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1. What test is appropriate for this problem? Does CLT apply?
The appropriate test is to test whether the rate of black applicants receiving a callback is less than the rate of white applicants. Since we have two populations (black and white applicants) we use a two-sample ttest. The Central Limit Theorem does apply since as we obtain more observations the sample mean will approach the true mean.

In [6]:
data.groupby('race').count().reset_index()

,race,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,2435,2435,2435,2435,2435,2435,2435,2435,2435,...,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435
1,w,2435,2435,2435,2435,2435,2435,2435,2435,2435,...,2435,2435,2435,2435,2435,2435,2435,2435,2435,2435


** Note that we have 2435 observations for both black and white applicants and no missing data in the race and call columns. **

In [7]:
black_applicants= data[data.race=='b']
white_applicants=data[data.race=='w']

# 2. What are the null and alternate hypotheses?
**Null Hypothesis: Call rate of black applicants = Call rate of white applicants **

**Alternative Hypothesis: Call rate of black applicants =/= Call rate of white applicants**

In [8]:
black_call_rate = black_applicants.call.mean()
white_call_rate = white_applicants.call.mean()
print("Black Call Rate: ", black_call_rate)
print("White Call Rate: ", white_call_rate)

Black Call Rate:  0.0644763857126236
White Call Rate:  0.09650924056768417


** We see that the sample call rate for  applicants with black sounding names  is 6.44% and the sample call rate for  applicants with white sounding names  is 9.65%.**

# 3. Compute margin of error, confidence interval, and p-value.

In [9]:
z,p=two_sample = stats.ttest_ind(black_applicants.call, white_applicants.call, equal_var=False)

cm = sms.CompareMeans(sms.DescrStatsW(black_applicants.call), sms.DescrStatsW(white_applicants.call))
conf_low,conf_high =cm.tconfint_diff(usevar='unequal')
print("Margin of Error: ", (conf_high-conf_low)/2)
print("95% Confidence Interval: ", (conf_low,conf_high))
print("p-value: ",p)

Margin of Error:  0.0152621802255
95% Confidence Interval:  (-0.047295034434899373, -0.016770673983991798)
p-value:  3.94294151365e-05


# 4. Write a story describing the statistical significance in the context or the original problem.

** Since the p-value is less than 0.05 we have that there is a statistical significance that the call rate for applicants with black sounding names is less than that of applicants with white sounding names and we reject the null hypothesis. From our 95% confidence interval we have that applicants with white sounding names have betwen a 1.7-4.7% better chance of getting a call back than applicants with a black sounding name.** 

#  5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

** From our analysis we can not conclude that race/na1me is the most important factor in callback success because it is the only factor we analyzed. There are other factors such as education and work experience that one would expect would have a larger impact on the callback rate. To amend our analysis we would have to take these variables into account (and possibly others) to obtain a clearer picture of what is the most important feature. **